### Import dependencies

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import socket
import pandas as pd
import pdfplumber
import os
from tqdm import tqdm
import xlrd
import xlwt
from pdf2image import convert_from_path
import pytesseract
import time


### Load data search file and create DF / .csv for final formatting

In [2]:
# Load the combined search output CSV file
df = pd.read_csv('Investment Research/Investment_Research_Test1.csv')

# Clean the date column to remove any extra characters or spaces
df['Date'] = df['Date'].str.strip()

# Define a function to parse dates with various formats
def parse_date(date_str):
    for fmt in ('%d-%b-%y', '%d-%b-%Y', '%d-%B-%y', '%d-%B-%Y'):
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    raise ValueError(f"Date format not recognized: {date_str}")

# Apply the date parsing function to the Date column
df['Date'] = df['Date'].apply(parse_date)

# Extract the month, day, and year from the date column
df['Month'] = df['Date'].dt.strftime('%b')
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year

# Create the new Join_Key column
df['Join_Key'] = df['Headline2'] + '_' + df['Month'] + '_' + df['Day'].astype(str) + ',_' + df['Year'].astype(str)

# Display the resulting DataFrame
print(df.head())

# Save the DataFrame to a new CSV file
df.to_csv('Investment Research/Investment_Research_Test3_Formatted.csv', index=False)

            Contributor              Analyst              Date/Time Company  \
0  CFRA Equity Research  Handshoe, Jonnathan  May 16, 2024 10:10 PM     MRO   
1  CFRA Equity Research    Glickman, Stewart  May 14, 2024 10:02 PM     EOG   
2  CFRA Equity Research    Glickman, Stewart  May 11, 2024 06:08 PM     EOG   
3  CFRA Equity Research    Glickman, Stewart  May 11, 2024 05:57 PM     DVN   
4  CFRA Equity Research    Glickman, Stewart  May 07, 2024 10:03 PM     COP   

                   Headline  Pages       Date               Headline2  \
0  Marathon Oil Corporation      9 2024-05-16  MarathonOilCorporation   
1       EOG Resources, Inc.      9 2024-05-14       EOGResources,Inc.   
2       EOG Resources, Inc.      9 2024-05-11       EOGResources,Inc.   
3  Devon Energy Corporation      9 2024-05-11  DevonEnergyCorporation   
4            ConocoPhillips      9 2024-05-07          ConocoPhillips   

   Unnamed: 8 Month  Day  Year                             Join_Key  
0         NaN   

### Extract text from directory with pdf analyst reports

In [3]:
# Function to extract text and title with OCR
def extract_text_and_title_with_ocr(pdf_path):
    try:
        # Convert PDF to images
        pages = convert_from_path(pdf_path, 300)
        text = ""
        for page in pages:
            # Convert each page to text
            page_text = pytesseract.image_to_string(page, lang='eng')
            text += page_text + "\n"
        return text
    except Exception as e:
        print("Error reading " + pdf_path + ": " + str(e))
        return None

# Function to extract date from filename
def extract_date_from_filename(filename):
    date_pattern = r'(\w{3})_(\d{1,2}),_(\d{4})\.pdf'
    match = re.search(date_pattern, filename)
    if match:
        return match.group(1) + " " + match.group(2) + ", " + match.group(3)
    else:
        return 'Date not found'

# Function to extract document ID
def extract_document_id(title):
    doc_id_pattern = r'(\d+)[^_]*'
    match = re.search(doc_id_pattern, title)
    if match:
        return match.group(1)
    else:
        return 'ID not found'

# Function to extract join key from filename
def extract_join_key(filename):
    join_key_pattern = r'_(.*?)\.pdf'
    match = re.search(join_key_pattern, filename)
    if match:
        return match.group(1)
    else:
        return 'Join_Key not found'

# Directory containing PDFs
pdf_dir = 'Investment Research/PDFs_All_V2'
pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]

# Start time
start_time = time.time()

# Extract data from each PDF
data = []
total_pdfs = len(pdf_files)
for i, pdf_file in enumerate(pdf_files):
    pdf_path = os.path.join(pdf_dir, pdf_file)
    text = extract_text_and_title_with_ocr(pdf_path)
    if text is None:
        continue  # Skip files that couldn't be read
    title = pdf_file  # Use file name as title
    source = pdf_file.split('_')[0]  # Extract source from file name
    date = extract_date_from_filename(pdf_file)  # Extract date from filename
    unique_id = extract_document_id(title)
    join_key = extract_join_key(pdf_file)  # Extract join key from filename
    data.append({'Unique_ID': unique_id, 'Join_Key': join_key, 'Date': date, 'Title': title, 'Source': source, 'Text': text})

    # Print progress and save intermediate CSV every 5 PDFs
    if (i + 1) % 5 == 0 or (i + 1) == total_pdfs:
        elapsed_time = time.time() - start_time
        print(f"Processed {i + 1} of {total_pdfs} PDFs. Elapsed time: {elapsed_time:.2f} seconds.")

        # Create a DataFrame for intermediate data
        df = pd.DataFrame(data)

        # Ensure Join_Key is a string and clean it
        def clean_join_key(key):
            key = re.sub(r'[^\w\s]', '', key.lower())
            key = re.sub(r'_(0)', '_', key)  # Remove '0' following the second underscore
            return key

        df['Join_Key'] = df['Join_Key'].astype(str).apply(clean_join_key)

        # Save intermediate DataFrame to CSV
        intermediate_csv_path = f'Investment Research/Intermediate_DF.csv'
        df.to_csv(intermediate_csv_path, index=False)
        print(f"Intermediate data saved to {intermediate_csv_path}")

Processed 5 of 4846 PDFs. Elapsed time: 173.08 seconds.
Intermediate data saved to Investment Research/Intermediate_5_DF.csv
Processed 10 of 4846 PDFs. Elapsed time: 329.21 seconds.
Intermediate data saved to Investment Research/Intermediate_10_DF.csv
Processed 15 of 4846 PDFs. Elapsed time: 502.76 seconds.
Intermediate data saved to Investment Research/Intermediate_15_DF.csv
Processed 20 of 4846 PDFs. Elapsed time: 658.33 seconds.
Intermediate data saved to Investment Research/Intermediate_20_DF.csv
Processed 25 of 4846 PDFs. Elapsed time: 827.88 seconds.
Intermediate data saved to Investment Research/Intermediate_25_DF.csv
Processed 30 of 4846 PDFs. Elapsed time: 974.07 seconds.
Intermediate data saved to Investment Research/Intermediate_30_DF.csv
Processed 35 of 4846 PDFs. Elapsed time: 1098.95 seconds.
Intermediate data saved to Investment Research/Intermediate_35_DF.csv
Processed 40 of 4846 PDFs. Elapsed time: 1266.56 seconds.
Intermediate data saved to Investment Research/Interme

Processed 325 of 4846 PDFs. Elapsed time: 10169.21 seconds.
Intermediate data saved to Investment Research/Intermediate_325_DF.csv
Processed 330 of 4846 PDFs. Elapsed time: 10304.39 seconds.
Intermediate data saved to Investment Research/Intermediate_330_DF.csv
Processed 335 of 4846 PDFs. Elapsed time: 10452.55 seconds.
Intermediate data saved to Investment Research/Intermediate_335_DF.csv
Processed 340 of 4846 PDFs. Elapsed time: 10625.67 seconds.
Intermediate data saved to Investment Research/Intermediate_340_DF.csv
Processed 345 of 4846 PDFs. Elapsed time: 10791.46 seconds.
Intermediate data saved to Investment Research/Intermediate_345_DF.csv
Processed 350 of 4846 PDFs. Elapsed time: 10940.34 seconds.
Intermediate data saved to Investment Research/Intermediate_350_DF.csv
Processed 355 of 4846 PDFs. Elapsed time: 11089.59 seconds.
Intermediate data saved to Investment Research/Intermediate_355_DF.csv
Processed 360 of 4846 PDFs. Elapsed time: 11259.97 seconds.
Intermediate data saved

KeyboardInterrupt: 

### Clean and merge with data search file to create final DF

In [ ]:

# Ensure Join_Key is a string and clean it
df['Join_Key'] = df['Join_Key'].astype(str).apply(clean_join_key)

# Save to CSV for new reference file
csv_path = 'Investment Research/Investment_Research_Test3_DF.csv'
df.to_csv(csv_path, index=False)
print('Data saved to ' + csv_path)

# Load existing CSV from search .csv
existing_csv_path = 'Investment Research/Investment_Research_Test3_Formatted.csv'
existing_df = pd.read_csv(existing_csv_path)

# Ensure Join_Key is a string and clean it in the existing DataFrame
existing_df['Join_Key'] = existing_df['Join_Key'].astype(str).apply(clean_join_key)

# Merge the 'Text' column from new data to the existing DataFrame based on 'Join_Key'
merged_df = existing_df.merge(df[['Join_Key', 'Text']], on='Join_Key', how='left')

# Select only the desired columns
columns_to_keep = ['Contributor', 'Date/Time', 'Date', 'Company', 'Headline', 'Text']
final_df = merged_df[columns_to_keep]

# Replace values in the "Company" column
replacements = {
    "Pioneer Natural Resources Company": "PXD",
    "Concho Resources Inc.": "CXO",
    "BP.": "BP",
    "PDC Energy, Inc.": "PDCE"
}

final_df['Company'] = final_df['Company'].replace(replacements)

# Rename the "Company" column to "Ticker"
final_df.rename(columns={"Company": "Ticker"}, inplace=True)

# Save to CSV
csv_path2 = 'Investment Research/Investment_Research_Test3_Final.csv'
final_df.to_csv(csv_path2, index=False)
print('Updated data saved to ' + csv_path2)

In [ ]:
final_df

In [ ]:
# Load the CSV file
csv_path2 = 'Investment Research/Investment_Research_Test3_Final.csv'
df = pd.read_csv(csv_path2)

# Function to truncate text at "Glossary" or "Analyst Research Notes and other Company News"
def truncate_text_cf(text):
    if isinstance(text, str):
        glossary_index = text.find("Glossary")
        analyst_notes_index = text.find("Analyst Research Notes and other Company News")
        
        # Find the earliest occurrence of either string
        if glossary_index != -1 and analyst_notes_index != -1:
            truncate_index = min(glossary_index, analyst_notes_index)
        elif glossary_index != -1:
            truncate_index = glossary_index
        elif analyst_notes_index != -1:
            truncate_index = analyst_notes_index
        else:
            truncate_index = -1
        
        if truncate_index != -1:
            return text[:truncate_index]
    return text

# Function to truncate text at "Argus Research Disclaimer"
def truncate_text_argus(text):
    if isinstance(text, str):
        argus_disclaimer_index = text.lower().find("argus research disclaimer")
        if argus_disclaimer_index != -1:
            return text[:argus_disclaimer_index]
    return text

# Apply the function to rows where "Contributor" contains "CFRA"
df.loc[df['Contributor'].str.contains('CFRA', na=False, case=False), 'Text'] = df.loc[df['Contributor'].str.contains('CFRA', na=False, case=False), 'Text'].apply(truncate_text_cf)

# Apply the function to rows where "Contributor" contains "Argus" (case insensitive)
df.loc[df['Contributor'].str.contains('Argus', na=False, case=False), 'Text'] = df.loc[df['Contributor'].str.contains('Argus', na=False, case=False), 'Text'].apply(truncate_text_argus)

# Save the modified dataframe to a new CSV file
new_csv_path = csv_path2.replace('.csv', '_Trimmed.csv')
df.to_csv(new_csv_path, index=False)

print("File saved as " + new_csv_path)

In [ ]:
# Load the CSV file
csv_path2 = 'Investment Research/Investment_Research_Test3_Final_Trimmed.csv'
df = pd.read_csv(csv_path2)

# Function to find the nearest closing quote around the midpoint
def find_nearest_closing_quote(df, midpoint):
    # Search for the closing quote in the "Text" column
    closing_quotes = df['Text'].str.endswith('"', na=False)
    
    # Find the index of the nearest closing quote before or after the midpoint
    before_midpoint = closing_quotes[:midpoint][::-1].idxmax()
    after_midpoint = closing_quotes[midpoint:].idxmax() + midpoint
    
    # Choose the nearest one
    if midpoint - before_midpoint <= after_midpoint - midpoint:
        return before_midpoint + 1  # +1 to include the closing quote line
    else:
        return after_midpoint + 1  # +1 to include the closing quote line

# Calculate the midpoint of the dataframe
midpoint = len(df) // 2

# Find the nearest closing quote around the midpoint
split_point = find_nearest_closing_quote(df, midpoint)

# Split the dataframe into two halves
df1 = df.iloc[:split_point]
df2 = df.iloc[split_point:]

# Save each half to a new CSV file
csv_path1 = csv_path2.replace('.csv', '_1.csv')
csv_path2 = csv_path2.replace('.csv', '_2.csv')

df1.to_csv(csv_path1, index=False)
df2.to_csv(csv_path2, index=False)

print("Files saved as " + csv_path1 + " and " + csv_path2)